### Creating of Final Data CSV Files and Final Pickle Files
- The first large loop is a data processing loop that runs some cleaning functions, previously devised in notebook 3.5 and additionally adjusts the format of the columns so that the dataframe would be smaller and hence make the pickle files smaller.
- The large second loop creates the pickle files on this processed data and prints out some evaluation metrics for each of the bus lines.
- The average MAPE is approximately 14.7% and the average R2 is approximately 0.923

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report
import seaborn as sns
from scipy import stats
from pathlib import Path
import pickle
import numpy as np
import math
import time

import gc
import psutil
import multiprocessing
#from multiprocessing import Pool
import os
import signal
from os import getpid
from sys import argv, exit

In [2]:
psutil.virtual_memory()

svmem(total=67481169920, available=23789481984, percent=64.7, used=34939871232, free=17533669376, active=32734658560, inactive=14866296832, buffers=1870868480, cached=13136760832, shared=8072048640, slab=1610731520)

In [3]:
psutil.users()

[suser(name='team9', terminal='pts/32', host='95.44.114.87', started=1660256896.0, pid=18250),
 suser(name='team10', terminal='pts/31', host='tmux(22475).%0', started=1657274368.0, pid=22475),
 suser(name='team1', terminal='pts/45', host='37.228.226.113', started=1660251136.0, pid=21931),
 suser(name='team9', terminal='pts/46', host='95.44.114.87', started=1660257024.0, pid=18765)]

In [4]:
cd tmp

/home/team9/tmp


In [5]:
cd data 

/home/team9/tmp/data


In [93]:
rm -rf pickle_files

In [94]:
mkdir pickle_files

In [8]:
#cd ..

In [9]:
#cd ..

In [95]:
ls

46A_COPY_MADE_FOR_XING_TO_CHECK.pickle
ben_models/
Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
final_data/
leavetimes/
lines_no_outliers/
mesh_xing_ben_victoria-Copy1.ipynb
mesh_xing_ben_victoria.ipynb
pickle_files/
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
test_46A_copy.pkl
test_46a_dask_2.csv
test_46a_dask.csv
test_46a_max_values.pkl
test_46a_min_estimators.pkl
test_46A_v_2.pkl
test_46A_victoria.pkl
test_pickle_ben.pickle
test_pickle_ben.pkl
test_pickle_victoria.pkl
trips_lines_ready.csv
trips_modelling.csv
trips_ready.csv
weather_factorised.csv
weather_ready.csv
weather_trips.csv


### Note that hours start at 0(for 24), days start at 0(Monday) and months start at 1(January)

- Rush hour is 7am-9am on work days

In [49]:
def cal_rush(df):
    df['rush_hour'] = 0
    rush_hour = [7,8,9,16,17,18,19]
    weekends = [5,6]
    for i in rush_hour:
        df['rush_hour'].loc[df['HOUR'] == i] = 1
    for j in weekends:
        df['rush_hour'].loc[df['DAYOFWEEK'] == j] = 0
    return df


- Friday and Saturdays grouped together because both are party nights

In [50]:
def cal_frisat(df):
    df['frisat'] =0
    weekends = [4, 5]
    for i in weekends:
        df['frisat'].loc[df['DAYOFWEEK'] == i] = 1
    return df

- Late nights are between 8pm and midnight. After midnight is seperate because many buses don't run then so it would be wrong to group.

In [51]:
def cal_LATE_NIGHT(df):
    df['LATE_NIGHT'] = 0
    LATE_NIGHT = [20, 21, 22, 23, 0]
    for i in LATE_NIGHT:
        df['LATE_NIGHT'].loc[df['HOUR'] == i] = 1
    return df

- Midday is between 10am and 3pm. By adding this feature it also distinguishes the very early hours of the morning by having a zero for all time columns.

In [99]:
def cal_MIDDAY(df):
    df['MIDDAY'] = 0
    MIDDAY = [10, 11, 12, 13, 14, 15]
    weekends = [5, 6]
    for i in MIDDAY:
        df['MIDDAY'].loc[df['HOUR'] == i] = 1

    return df

- Afternoon is between 1pm and 7pm every day

In [116]:
def cal_afternoon(df):
    df['AFTERNOON'] = 0
    AFTERNOON = [13, 14, 15, 16, 17, 18, 19]
    for i in AFTERNOON:
        df['AFTERNOON'].loc[df['HOUR'] == i] = 1

    return df

- Weekdays are from Monday to Friday. Note that Friday is included but can be distinguished as different because it is also in FriSat column. Sunday is distinguished becaused it is not in frisat or weekdays.

In [53]:
def cal_midweek(df):
    df['WEEKDAY'] = 0
    WEEKDAY = [0, 1, 2, 3, 4]
    for i in WEEKDAY:
        df['WEEKDAY'].loc[df['DAYOFWEEK'] == i] = 1
    return df

- Summer Months are June July and August as those are the months when most students are not in school so route changes occur.

In [54]:
def cal_summer(df):
    df['SUMMER'] = 0
    summer = [6,7, 8]
    for i in summer:
        df['SUMMER'].loc[df['MONTH'] == i] = 1
    return df

- Winter is  the months of November, December, January and February because this when the coldest weather is. Note that no seperate columns were created for Autumn and Spring because they have similar weather and students will stillbe in school for both.

In [55]:
def cal_WINTER(df):
    df['WINTER'] = 0
    WINTER = [11, 12, 1, 2]
    for i in WINTER:
        df['WINTER'].loc[df['MONTH'] == i] = 1
    return df

- Morning column is for times between 5am and 12(midday). It is just a way of further segmenting time segments so that the midday column doesnt group 10am with 3pm and so that 5am and 6am are not grouped with the latest hours of the night as many buses are in service at this time that are not busy late at night. Rush hour column will also split this column so that 5am and 6am are not grouped with the busier hours.

In [56]:
def cal_MORNING(df):
    df['MORNING'] = 0
    MORNING = [5, 6, 7, 8, 9, 10, 11, 12]
    for i in MORNING:
        df['MORNING'].loc[df['HOUR'] == i] = 1
    return df

In [57]:
def divide_60(x):
    x = x/60
    return x

In [20]:
#cd 

In [108]:
mkdir final_data

In [132]:
ls

46A_COPY_MADE_FOR_XING_TO_CHECK.pickle
ben_models/
Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
final_data/
leavetimes/
lines_no_outliers/
mesh_xing_ben_victoria-Copy1.ipynb
mesh_xing_ben_victoria.ipynb
pickle_files/
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
test_46A_copy.pkl
test_46a_dask_2.csv
test_46a_dask.csv
test_46a_max_values.pkl
test_46a_min_estimators.pkl
test_46A_v_2.pkl
test_46A_victoria.pkl
test_pickle_ben.pickle
test_pickle_ben.pkl
test_pickle_victoria.pkl
trips_lines_ready.csv
trips_modelling.csv
trips_ready.csv
weather_factorised.csv
weather_ready.csv
weather_trips.csv


In [23]:
#mkdir final_data

mkdir: cannot create directory ‘final_data’: File exists


In [23]:
'102_1', '102_2', '104_1', '104_2', '111_1', '111_2', '114_1', '114_2', '116_1', '116_2', '118_2', '11_1', '11_2', '120_1', '120_2', '122_1', '122_2', '123_1', '123_2', '130_1', '130_2', '13_1', '13_2', '140_1', '140_2', '142_1', '142_2', '145_1', '145_2', '14C_1', '14C_2', '14_1', '14_2', '150_1', '150_2', '151_1', '151_2', '15A_1', '15A_2', '15B_1', '15B_2', '15D_1', '15D_2', '15_1', '15_2', '161_1', '161_2', '16C_1', '16C_2', '16D_1', '16_1', '16_2', '17A_1', '17A_2', '17_1', '17_2', '184_1', '184_2', '185_1', '185_2', '18_1', '18_2', '1_1', '1_2', '220_1', '220_2', '236_1', '236_2', '238_1', '238_2', '239_1', '239_2', '25A_1', '25A_2', '25B_1', '25B_2', '25D_1', '25D_2', '25X_1', '25X_2', '25_1', '25_2', '26_1', '26_2', '270_1', '270_2', '27A_1', '27A_2', '27B_1', '27B_2', '27X_1', '27X_2', '27_1', '27_2', '29A_1', '29A_2', '31A_1', '31A_2', '31B_1', '31B_2', '31D_1', '31D_2', '31_1', '31_2', '32X_1', '32X_2', '32_1', '32_2', '33A_1', '33A_2', '33B_1', '33B_2', '33D_1', '33D_2', '33E_1', '33X_1', '33X_2', '33_1', '33_2', '37_1', '37_2', '38A_1', '38A_2', '38B_1', '38B_2', '38D_1', '38D_2', '38_1', '38_2', '39A_1', '39A_2', '39X_1', '39X_2', '39_1', '39_2', '40B_1', '40B_2', '40D_1', '40D_2', '40E_1', '40E_2', '40_1', '40_2', '41A_2', '41B_1', '41B_2', '41C_1', '41C_2', '41D_1', '41D_2', '41X_1', '41X_2', '41_1', '41_2', '42D_1', '42D_2', '42_1', '42_2', '43_1', '43_2', '44B_1', '44B_2', '44_1', '44_2', '45A_1', '45A_2', '46A_1', '46A_2', '46E_2', '47_1', '47_2', '49_1', '49_2', '4_1', '4_2', '51D_1', '51D_2', '51X_2', '53_1', '53_2', '54A_1', '54A_2', '56A_1', '56A_2', '59_1', '59_2', '61_1', '61_2', '63_1', '63_2', '65B_1', '65B_2', '65_1', '65_2', '66A_1', '66A_2', '66B_1', '66B_2', '66X_1', '66X_2', '66_1', '66_2', '67X_1', '67X_2', '67_1', '67_2', '68A_1', '68A_2', '68X_2', '68_1', '68_2', '69X_1', '69X_2', '69_1', '69_2', '70D_1', '70D_2', '70_1', '70_2', '75_1', '75_2', '76A_1', '76A_2', '76_1', '76_2', '77A_1', '77A_2', '77X_2', '79A_1', '79A_2'

('102_1',
 '102_2',
 '104_1',
 '104_2',
 '111_1',
 '111_2',
 '114_1',
 '114_2',
 '116_1',
 '116_2',
 '118_2',
 '11_1',
 '11_2',
 '120_1',
 '120_2',
 '122_1',
 '122_2',
 '123_1',
 '123_2',
 '130_1',
 '130_2',
 '13_1',
 '13_2',
 '140_1',
 '140_2',
 '142_1',
 '142_2',
 '145_1',
 '145_2',
 '14C_1',
 '14C_2',
 '14_1',
 '14_2',
 '150_1',
 '150_2',
 '151_1',
 '151_2',
 '15A_1',
 '15A_2',
 '15B_1',
 '15B_2',
 '15D_1',
 '15D_2',
 '15_1',
 '15_2',
 '161_1',
 '161_2',
 '16C_1',
 '16C_2',
 '16D_1',
 '16_1',
 '16_2',
 '17A_1',
 '17A_2',
 '17_1',
 '17_2',
 '184_1',
 '184_2',
 '185_1',
 '185_2',
 '18_1',
 '18_2',
 '1_1',
 '1_2',
 '220_1',
 '220_2',
 '236_1',
 '236_2',
 '238_1',
 '238_2',
 '239_1',
 '239_2',
 '25A_1',
 '25A_2',
 '25B_1',
 '25B_2',
 '25D_1',
 '25D_2',
 '25X_1',
 '25X_2',
 '25_1',
 '25_2',
 '26_1',
 '26_2',
 '270_1',
 '270_2',
 '27A_1',
 '27A_2',
 '27B_1',
 '27B_2',
 '27X_1',
 '27X_2',
 '27_1',
 '27_2',
 '29A_1',
 '29A_2',
 '31A_1',
 '31A_2',
 '31B_1',
 '31B_2',
 '31D_1',
 '31D_2',
 '31

In [24]:
102_1

1021

In [96]:
lines = ["102_1",'102_2', '104_1', '104_2', '111_1', '111_2', '114_1', '114_2', '116_1', '116_2', '118_2', '11_1', '11_2', '120_1', '120_2', '122_1', '122_2', '123_1', '123_2', '130_1', '130_2', '13_1', '13_2', '140_1', '140_2', '142_1', '142_2', '145_1', '145_2', '14C_1', '14C_2', '14_1', '14_2', '150_1', '150_2', '151_1', '151_2', '15A_1', '15A_2', '15B_1', '15B_2', '15D_1', '15D_2', '15_1', '15_2', '161_1', '161_2', '16C_1', '16C_2', '16D_1', '16_1', '16_2', '17A_1', '17A_2', '17_1', '17_2', '184_1', '184_2', '185_1', '185_2', '18_1', '18_2', '1_1', '1_2', '220_1', '220_2', '236_1', '236_2', '238_1', '238_2', '239_1', '239_2', '25A_1', '25A_2', '25B_1', '25B_2', '25D_1', '25D_2', '25X_1', '25X_2', '25_1', '25_2', '26_1', '26_2', '270_1', '270_2', '27A_1', '27A_2', '27B_1', '27B_2', '27X_1', '27X_2', '27_1', '27_2', '29A_1', '29A_2', '31A_1', '31A_2', '31B_1', '31B_2', '31D_1', '31D_2', '31_1', '31_2', '32X_1', '32X_2', '32_1', '32_2', '33A_1', '33A_2', '33B_1', '33B_2', '33D_1', '33D_2', '33E_1', '33X_1', '33X_2', '33_1', '33_2', '37_1', '37_2', '38A_1', '38A_2', '38B_1', '38B_2', '38D_1', '38D_2', '38_1', '38_2', '39A_1', '39A_2', '39X_1', '39X_2', '39_1', '39_2', '40B_1', '40B_2', '40D_1', '40D_2', '40E_1', '40E_2', '40_1', '40_2', '41A_2', '41B_1', '41B_2', '41C_1', '41C_2', '41D_1', '41D_2', '41X_1', '41X_2', '41_1', '41_2', '42D_1', '42D_2', '42_1', '42_2', '43_1', '43_2', '44B_1', '44B_2', '44_1', '44_2', '45A_1', '45A_2', '46A_1', '46A_2', '46E_2', '47_1', '47_2', '49_1', '49_2', '4_1', '4_2', '51D_1', '51D_2', '51X_2', '53_1', '53_2', '54A_1', '54A_2', '56A_1', '56A_2', '59_1', '59_2', '61_1', '61_2', '63_1', '63_2', '65B_1', '65B_2', '65_1', '65_2', '66A_1', '66A_2', '66B_1', '66B_2', '66X_1', '66X_2', '66_1', '66_2', '67X_1', '67X_2', '67_1', '67_2', '68A_1', '68A_2', '68X_2', '68_1', '68_2', '69X_1', '69X_2', '69_1', '69_2', '70D_1', '70D_2', '70_1', '70_2', '75_1', '75_2', '76A_1', '76A_2', '76_1', '76_2', '77A_1', '77A_2', '77X_2', '79A_1', '79A_2', '79_1', '79_2', '7A_1', '7A_2', '7B_1', '7B_2', '7D_1', '7D_2', '7_1', '7_2', '83A_1', '83A_2', '83_1', '83_2', '84A_1', '84A_2', '84X_1', '84X_2', '84_1', '84_2', '9_1', '9_2']

In [35]:
df = pd.read_csv(f'./114_1.csv')

In [134]:
cd ..

/home/team9/tmp/data


In [140]:
test46 = pd.read_csv(f'./lines_no_outliers/line_46A_2.csv')

In [141]:
test46

,Unnamed: 0,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID_DIRECTION,JOURNEY_TIME,HOUR,temp,wind_speed,weather_main,DAYOFWEEK,MONTH,error
0,69044389,7317634,58,801,40204,40685,229,46A_2,4110,11,13,6,1,4,8,481
1,69044390,7316161,58,801,66223,67036,229,46A_2,4990,19,17,6,0,4,8,813
2,98141396,8119478,1,2039,63120,63315,327,46A_2,0,18,8,4,0,4,11,195
3,98141545,8126084,28,2070,40367,40378,327,46A_2,1757,11,7,6,1,4,11,11
4,98141477,8119677,15,2058,22346,22413,327,46A_2,476,6,6,4,0,4,11,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023729,62252716,7181477,47,320,39898,40748,207,46A_2,3621,11,23,6,0,3,7,850
2023730,54103098,7104494,12,4570,74860,74903,180,46A_2,453,21,17,4,0,4,6,43
2023731,104885710,8466570,26,2068,30323,30252,351,46A_2,1675,8,5,4,0,0,12,-71
2023732,104885711,8466570,27,2069,30418,30333,351,46A_2,1756,8,5,4,0,0,12,-85


In [41]:
df.value_counts()

PROGRNUMBER  JOURNEY_TIME  weather_main  rush_hour  LATE_NIGHT  WEEKDAY  SUMMER  WINTER  MIDDAY  frisat  MORNING
2            1             0             1          0           1        0       0       0       0       1          229
                                         0          0           1        0       0       1       0       1          174
                                         1          0           1        0       0       0       0       0          168
                                         0          0           1        0       0       1       0       0          164
                                         1          0           1        1       0       0       0       1          159
                                                                                                                   ... 
19           16            0             0          1           0        0       0       0       0       0            1
                                               

In [28]:
bad_f = f.loc[f.JOURNEY_TIME < 10]

In [29]:
bad_f.PROGRNUMBER.value_counts()

1     17516
2       163
3         5
4         3
9         1
20        1
24        1
23        1
22        1
21        1
26        1
25        1
18        1
16        1
17        1
19        1
Name: PROGRNUMBER, dtype: int64

In [89]:
df

,PROGRNUMBER,JOURNEY_TIME,weather_main,rush_hour,LATE_NIGHT,WEEKDAY,SUMMER,WINTER,MIDDAY,frisat,MORNING
0,48,42,0,0,0,1,0,0,1,1,1
1,27,17,0,1,0,1,0,0,0,0,1
2,26,16,0,1,0,1,0,0,0,0,1
3,25,16,0,1,0,1,0,0,0,0,1
4,24,15,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
449781,22,21,0,1,0,1,0,0,0,0,0
449782,29,20,0,0,1,1,0,0,0,0,0
449783,30,22,0,0,1,1,0,0,0,0,0
449784,28,23,0,0,0,1,0,0,1,0,1


In [102]:
mkdir final_data

In [115]:
ls 

46A_COPY_MADE_FOR_XING_TO_CHECK.pickle
ben_models/
Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
final_data/
leavetimes/
lines_no_outliers/
mesh_xing_ben_victoria-Copy1.ipynb
mesh_xing_ben_victoria.ipynb
pickle_files/
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
test_46A_copy.pkl
test_46a_dask_2.csv
test_46a_dask.csv
test_46a_max_values.pkl
test_46a_min_estimators.pkl
test_46A_v_2.pkl
test_46A_victoria.pkl
test_pickle_ben.pickle
test_pickle_ben.pkl
test_pickle_victoria.pkl
trips_lines_ready.csv
trips_modelling.csv
trips_ready.csv
weather_factorised.csv
weather_ready.csv
weather_trips.csv


- Test run outside of the loop

In [142]:
line = "46A_2"
df = pd.read_csv(f'./lines_no_outliers/line_{line}.csv')
#df = df.drop("Unnamed: 0", axis=1)
# shrinking df size
df["DOY"] = df["DOY"].astype("int16")
df["TRIPID"] = df["TRIPID"].astype("int32")
df["PROGRNUMBER"] = df["PROGRNUMBER"].astype("int8")
df["STOPPOINTID"] = df["STOPPOINTID"].astype("int16")

df["PLANNEDTIME_ARR"] = df["PLANNEDTIME_ARR"].astype("int32")

df["ACTUALTIME_ARR"] = df["ACTUALTIME_ARR"].astype("int32")
df["LINEID_DIRECTION"] = df["LINEID_DIRECTION"].astype("category")
# bigger here for division later
df.JOURNEY_TIME = df.JOURNEY_TIME.astype("float64")
df["HOUR"] = df["HOUR"].astype("int8")
df["MONTH"] = df["MONTH"].astype("int8")
df["error"] = df["error"].astype("int32")

df["DAYOFWEEK"] = df["DAYOFWEEK"].astype("int8")
df["temp"] = df["temp"].astype("int8")
df["wind_speed"] = df["wind_speed"].astype("int8")
df["weather_main"] = df["weather_main"].astype("category")

# dropping first stops
df = df.loc[df.PROGRNUMBER != 1]

# removing probable bad routes
if line not in weird_routes :
    minimum = 10
else:
    minimun = 0

negs = df.loc[df.JOURNEY_TIME < 0]
neggg = negs.groupby(['TRIPID','DOY']).size().reset_index().rename(columns={0:'count'})
neglist = neggg.TRIPID.tolist()
negDOY = neggg.DOY.tolist()
for T in range(0, len(neglist)):
    df = df.loc[(df.TRIPID != neglist[T]) & (df.DOY != negDOY[T])]

# high outlier removal
df = df[df.JOURNEY_TIME < df.JOURNEY_TIME.quantile(.999)]

#removing non-credibly short routes
df = df.loc[df['JOURNEY_TIME']> minimum]


#save
df.to_csv(f'./XING_TEST_{line}.csv', index=False)
print(line)
print(count)
print(totalcount)

46A_2
252
252


- These are routes which I identified as having extremely close first stops so are treated differently with the cleaning of data.

In [26]:
weird_routes = ['116_1', '116_2', '123_1', '13_1', '161_1', '161_2', '239_2', '37_1', '39A_2', '39_2', '47_1']

In [48]:
ls

ben_models/
Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
final_data/
leavetimes/
lines_no_outliers/
mesh_xing_ben_victoria-Copy1.ipynb
mesh_xing_ben_victoria.ipynb
pickle_files/
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
test_46A_copy.pkl
test_46a_dask_2.csv
test_46a_dask.csv
test_46a_max_values.pkl
test_46a_min_estimators.pkl
test_46A_v_2.pkl
test_46A_victoria.pkl
test_pickle_ben.pickle
test_pickle_ben.pkl
test_pickle_victoria.pkl
trips_lines_ready.csv
trips_modelling.csv
trips_ready.csv
weather_factorised.csv
weather_ready.csv
weather_trips.csv


In [126]:
mkdir final_data

mkdir: cannot create directory ‘final_data’: File exists


In [127]:
%%time
count = 0
totalcount = 0
for line in lines:
    count+=1
    totalcount+=1
    
    #reading file
    df = pd.read_csv(f'./lines_no_outliers/line_{line}.csv')
    #df = df.drop("Unnamed: 0", axis=1)
    # shrinking df size
    df["DOY"] = df["DOY"].astype("int16")
    df["TRIPID"] = df["TRIPID"].astype("int32")
    df["PROGRNUMBER"] = df["PROGRNUMBER"].astype("int8")
    df["STOPPOINTID"] = df["STOPPOINTID"].astype("int16")

    df["PLANNEDTIME_ARR"] = df["PLANNEDTIME_ARR"].astype("int32")

    df["ACTUALTIME_ARR"] = df["ACTUALTIME_ARR"].astype("int32")
    df["LINEID_DIRECTION"] = df["LINEID_DIRECTION"].astype("category")
# bigger here for division later
    df.JOURNEY_TIME = df.JOURNEY_TIME.astype("float64")
    df["HOUR"] = df["HOUR"].astype("int8")
    df["MONTH"] = df["MONTH"].astype("int8")
    df["error"] = df["error"].astype("int32")

    df["DAYOFWEEK"] = df["DAYOFWEEK"].astype("int8")
    df["temp"] = df["temp"].astype("int8")
    df["wind_speed"] = df["wind_speed"].astype("int8")
    df["weather_main"] = df["weather_main"].astype("category")
    
    # dropping first stops
    df = df.loc[df.PROGRNUMBER != 1]
    
    # removing probable bad routes
    if line not in weird_routes :
        minimum = 10
    else:
        minimun = 0
        
    negs = df.loc[df.JOURNEY_TIME < 0]
    neggg = negs.groupby(['TRIPID','DOY']).size().reset_index().rename(columns={0:'count'})
    neglist = neggg.TRIPID.tolist()
    negDOY = neggg.DOY.tolist()
    for T in range(0, len(neglist)):
        df = df.loc[(df.TRIPID != neglist[T]) & (df.DOY != negDOY[T])]
        
    # high outlier removal
    df = df[df.JOURNEY_TIME < df.JOURNEY_TIME.quantile(.999)]
    
    #removing non-credibly short routes
    df = df.loc[df['JOURNEY_TIME']> minimum]
    
    # feature adjustments
    df = cal_rush(df)
    df = cal_LATE_NIGHT(df)
    df = cal_midweek(df)
    df = cal_summer(df)
    df = cal_WINTER(df)
    df = cal_MIDDAY(df)
    df = cal_frisat(df)
    df = cal_MORNING(df)
    df = cal_MIDDAY(df)
    #df = cal_afternoon(df)

    
    df["rush_hour"] = df["rush_hour"].astype("category")
    df["LATE_NIGHT"] = df["LATE_NIGHT"].astype("category")
    df["WEEKDAY"] = df["WEEKDAY"].astype("category")
    df["SUMMER"] = df["SUMMER"].astype("category")
    df["WINTER"] = df["WINTER"].astype("category")
    df["MIDDAY"] = df["MIDDAY"].astype("category")
    df["frisat"] = df["frisat"].astype("category")
    df["MORNING"] = df["MORNING"].astype("category")
   # df["AFTERNOON"] = df["AFTERNOON"].astype("category")
    
    # journey to minutes
   # df['JOURNEY_TIME']=df['JOURNEY_TIME'].apply(lambda x: divide_60(x))
    #df.JOURNEY_TIME = df['JOURNEY_TIME'].apply(np.ceil)
    df.JOURNEY_TIME = df.JOURNEY_TIME.astype("int32")
    
    df = df.drop(["Unnamed: 0","ACTUALTIME_ARR", "STOPPOINTID", "LINEID_DIRECTION", "error", "DAYOFWEEK", "MONTH","HOUR", "TRIPID", "temp", "wind_speed", "HOUR", "DOY", "DAYOFWEEK", "PLANNEDTIME_ARR"], axis=1)
    
    #save
    df.to_csv(f'./final_data/{line}.csv', index=False)
    print(line)
   # print(count)
    print(totalcount)
    #ram management
 #   if count == 50:
     #   gc.collect()
   #     %reset Out
    #    count = 0



/tmp/ipykernel_25281/1458245046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/1458245046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['DAYOFWEEK'] == j] = 0
/tmp/ipykernel_25281/4078674252.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LATE_NIGHT'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/2639058501.py:5: SettingWithCopyWarni

102_1
1


/tmp/ipykernel_25281/1458245046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/1458245046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['DAYOFWEEK'] == j] = 0
/tmp/ipykernel_25281/4078674252.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LATE_NIGHT'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/2639058501.py:5: SettingWithCopyWarni

102_2
2


/tmp/ipykernel_25281/1458245046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/1458245046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['DAYOFWEEK'] == j] = 0
/tmp/ipykernel_25281/4078674252.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LATE_NIGHT'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/2639058501.py:5: SettingWithCopyWarni

...

84_2
250


/tmp/ipykernel_25281/1458245046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/1458245046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['DAYOFWEEK'] == j] = 0
/tmp/ipykernel_25281/4078674252.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LATE_NIGHT'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/2639058501.py:5: SettingWithCopyWarni

9_1
251


/tmp/ipykernel_25281/1458245046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/1458245046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rush_hour'].loc[df['DAYOFWEEK'] == j] = 0
/tmp/ipykernel_25281/4078674252.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LATE_NIGHT'].loc[df['HOUR'] == i] = 1
/tmp/ipykernel_25281/2639058501.py:5: SettingWithCopyWarni

9_2
252
CPU times: user 13min 13s, sys: 1min 14s, total: 14min 28s
Wall time: 15min 4s


In [ ]:
# last processed line  was 79A_2

In [67]:
cd final_data

/home/team9/tmp/data/final_data


In [68]:
ls

102_1.csv  150_2.csv  25_1.csv   33_1.csv   41_2.csv   54A_1.csv  70_2.csv
102_2.csv  151_1.csv  25_2.csv   33_2.csv   41A_2.csv  54A_2.csv  70D_1.csv
104_1.csv  151_2.csv  25A_1.csv  33A_1.csv  41B_1.csv  56A_1.csv  70D_2.csv
104_2.csv  15_1.csv   25A_2.csv  33A_2.csv  41B_2.csv  56A_2.csv  7_1.csv
111_1.csv  15_2.csv   25B_1.csv  33B_1.csv  41C_1.csv  59_1.csv   7_2.csv
111_2.csv  15A_1.csv  25B_2.csv  33B_2.csv  41C_2.csv  59_2.csv   75_1.csv
11_1.csv   15A_2.csv  25D_1.csv  33D_1.csv  4_1.csv    61_1.csv   75_2.csv
11_2.csv   15B_1.csv  25D_2.csv  33D_2.csv  41D_1.csv  61_2.csv   76_1.csv
114_1.csv  15B_2.csv  25X_1.csv  33E_1.csv  41D_2.csv  63_1.csv   76_2.csv
114_2.csv  15D_1.csv  25X_2.csv  33X_1.csv  41X_1.csv  63_2.csv   76A_1.csv
116_1.csv  15D_2.csv  26_1.csv   33X_2.csv  41X_2.csv  65_1.csv   76A_2.csv
116_2.csv  161_1.csv  26_2.csv   37_1.csv   42_1.csv   65_2.csv   77A_1.csv
118_2.csv  161_2.csv  270_1.csv  37_2.csv   42_2.csv   65B_1.csv  77A_2.csv
1_1.csv    16_1.csv  

In [71]:
cd ..

/home/team9/tmp/data


In [128]:
ls

46A_COPY_MADE_FOR_XING_TO_CHECK.pickle
ben_models/
Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
final_data/
leavetimes/
lines_no_outliers/
mesh_xing_ben_victoria-Copy1.ipynb
mesh_xing_ben_victoria.ipynb
pickle_files/
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
test_46A_copy.pkl
test_46a_dask_2.csv
test_46a_dask.csv
test_46a_max_values.pkl
test_46a_min_estimators.pkl
test_46A_v_2.pkl
test_46A_victoria.pkl
test_pickle_ben.pickle
test_pickle_ben.pkl
test_pickle_victoria.pkl
trips_lines_ready.csv
trips_modelling.csv
trips_ready.csv
weather_factorised.csv
weather_ready.csv
weather_trips.csv


In [73]:
test = pd.read_csv(f'./final_data/41_1.csv')

In [75]:
test.value_counts()

PROGRNUMBER  JOURNEY_TIME  weather_main  rush_hour  LATE_NIGHT  WEEKDAY  SUMMER  WINTER  MIDDAY  frisat  MORNING
2            1             0             1          0           1        0       0       0       0       0          352
                                         0          0           1        0       0       1       0       0          348
                                         1          0           1        0       0       0       0       1          335
3            3             0             1          0           1        0       0       0       0       1          313
2            1             0             0          0           1        0       1       1       0       0          312
                                                                                                                   ... 
13           31            0             0          0           0        0       0       0       0       0            1
37           58            1             0     

In [ ]:
cd data

 #### should be in /home/team9/tmp/data

In [84]:
cd ..

/home/team9/tmp/data


In [7]:
ls

46A_COPY_MADE_FOR_XING_TO_CHECK.pickle
ben_models/
Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
final_data/
leavetimes/
lines_no_outliers/
mesh_xing_ben_victoria-Copy1.ipynb
mesh_xing_ben_victoria.ipynb
pickle_files/
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
test_46A_copy.pkl
test_46a_dask_2.csv
test_46a_dask.csv
test_46a_max_values.pkl
test_46a_min_estimators.pkl
test_46A_v_2.pkl
test_46A_victoria.pkl
test_pickle_ben.pickle
test_pickle_ben.pkl
test_pickle_victoria.pkl
trips_lines_ready.csv
trips_modelling.csv
trips_ready.csv
weather_factorised.csv
weather_ready.csv
weather_trips.csv
XING_TEST_46A_2.csv


In [129]:
%%time
count = 0 
for line in lines:
    count+=1
    
    #reading file
    dt = pd.read_csv(f'./final_data/{line}.csv')
    #dt = dt.drop("Unnamed: 0", axis =1)  
    
    dt["rush_hour"] = dt["rush_hour"].astype("category")
    dt["LATE_NIGHT"] = dt["LATE_NIGHT"].astype("category")
    dt["WEEKDAY"] = dt["WEEKDAY"].astype("category")
    dt["SUMMER"] = dt["SUMMER"].astype("category")
    dt["WINTER"] = dt["WINTER"].astype("category")
    dt["MIDDAY"] = dt["MIDDAY"].astype("category")
    dt["frisat"] = dt["frisat"].astype("category")
    dt["MORNING"] = dt["MORNING"].astype("category")
    dt.JOURNEY_TIME = dt.JOURNEY_TIME.astype("int32")
    dt.PROGRNUMBER = dt.PROGRNUMBER.astype("int16")
    
    MAPEs = []
    R2 = []
    #dt = df
    y=dt["JOURNEY_TIME"] 
    x=dt.drop('JOURNEY_TIME', axis=1)
    features=x.columns.values.tolist()
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
    rfc = RandomForestRegressor(n_estimators=30, n_jobs=-1, max_depth=34, min_samples_leaf=10, min_samples_split=15)
    rfc.fit(X_train, y_train)
    prediction = rfc.predict(X_test)
    #modelEvalucation(y_test, prediction)
    MAPEs += [metrics.mean_absolute_percentage_error(y_test, prediction)]
    R2 += [metrics.r2_score(y_test, prediction)]
    print('MAPE: ',MAPEs)
    print('R2: ', R2)
    
    #save pickles
    t= "./pickle_files/" + line +'.pickle'
    f = open(t,'wb')
    pickle.dump(rfc,f, pickle.HIGHEST_PROTOCOL)
    print(f"line {line} done")
    print(count)
    
    
    

MAPE:  [0.12528181273959862]
R2:  [0.961469085476246]
line 102_1 done
1
MAPE:  [0.11394040801969892]
R2:  [0.9474432270917401]
line 102_2 done
2
MAPE:  [0.12291163148776124]
R2:  [0.9495600713639386]
line 104_1 done
3
MAPE:  [0.11611984476847889]
R2:  [0.9316189756453892]
line 104_2 done
4
MAPE:  [0.125111841400461]
R2:  [0.9211725289268614]
line 111_1 done
5
MAPE:  [0.12573550465877986]
R2:  [0.9539111688417191]
line 111_2 done
6
MAPE:  [0.18666337281982537]
R2:  [0.8899311908202991]
line 114_1 done
7
MAPE:  [0.1675563734599499]
R2:  [0.8815113629009803]
line 114_2 done
8
MAPE:  [0.15386874915255022]
R2:  [0.9326332656692762]
line 116_1 done
9
MAPE:  [0.18227814338659235]
R2:  [0.959722861936133]
line 116_2 done
10
MAPE:  [0.12374992381680322]
R2:  [0.9490306113123599]
line 118_2 done
11
MAPE:  [0.13858091118639812]
R2:  [0.9221817042413175]
line 11_1 done
12
MAPE:  [0.14994564449339914]
R2:  [0.9339632228378667]
line 11_2 done
13
MAPE:  [0.1766405643650355]
R2:  [0.8512918753663453]


In [38]:
average_MAPE = sum(MAPEs)/len(MAPEs)
print(average_MAPE)

0.1472750024104822


In [39]:
average_R2 = sum(R2)/len(R2)
print(average_R2)

0.9238686368517686


In [8]:
test = pickle.load(open("./pickle_files/102_1.pickle",'rb'))

In [10]:
data = pd.read_csv("./final_data/102_1.csv")

In [11]:
data.head()

,PROGRNUMBER,JOURNEY_TIME,weather_main,rush_hour,LATE_NIGHT,WEEKDAY,SUMMER,WINTER,MIDDAY,frisat,MORNING
0,48,2474,0,0,0,1,0,0,1,1,1
1,27,965,0,1,0,1,0,0,0,0,1
2,26,921,0,1,0,1,0,0,0,0,1
3,25,909,0,1,0,1,0,0,0,0,1
4,24,844,0,1,0,1,0,0,0,0,1


In [45]:
dt = data.iloc[[3000]]

In [46]:
dt

,PROGRNUMBER,JOURNEY_TIME,weather_main,rush_hour,LATE_NIGHT,WEEKDAY,SUMMER,WINTER,MIDDAY,frisat,MORNING
3000,11,407,0,1,0,1,0,1,0,0,0


In [47]:
y=dt["JOURNEY_TIME"] 
x=dt.drop('JOURNEY_TIME', axis=1)

In [38]:
MAPEs = []
R2 = []
#dt = df
y=dt["JOURNEY_TIME"] 
x=dt.drop('JOURNEY_TIME', axis=1)

prediction = test.predict(x)
#modelEvalucation(y_test, prediction)
MAPEs += [metrics.mean_absolute_percentage_error(y, prediction)]
R2 += [metrics.r2_score(y, prediction)]
print('MAPE: ',MAPEs)
print('R2: ', R2)

MAPE:  [0.10601393986808458]
R2:  [nan]


/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [48]:
prediction = test.predict(x)

In [49]:
prediction

array([467.56415636])